Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [2]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)
df = df.append(pd.DataFrame.from_records(spam))

skipped 2248.2004-09-23.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt


C:\Users\garvi\AppData\Local\Temp\ipykernel_3652\217166261.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame.from_records(spam))


In [3]:
df

,name,content,category
0,0001.1999-12-10.farmer.ham.txt,Subject: christmas tree farm pictures\n,ham
1,0002.1999-12-13.farmer.ham.txt,"Subject: vastar resources , inc .\ngary , prod...",ham
2,0003.1999-12-14.farmer.ham.txt,Subject: Calpine daily gas nomination\n- calpi...,ham
3,0004.1999-12-14.farmer.ham.txt,Subject: re : issue\nfyi - see note below - al...,ham
4,0005.1999-12-14.farmer.ham.txt,Subject: meter 7268 nov allocation\nfyi .\n- -...,ham
...,...,...,...
1491,5163.2005-09-06.GP.spam.txt,Subject: our pro - forma invoice attached\ndiv...,spam
1492,5164.2005-09-06.GP.spam.txt,Subject: str _ rndlen ( 2 - 4 ) } { extra _ ti...,spam
1493,5167.2005-09-06.GP.spam.txt,Subject: check me out !\n61 bb\nhey derm\nbbbb...,spam
1494,5170.2005-09-06.GP.spam.txt,Subject: hot jobs\nglobal marketing specialtie...,spam


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [4]:
import re

def preprocessor(e):
    e = re.sub(r"\s+", " ",e)
    e = e.lower()
    e = re.sub(r"[!@#$%^&*();<>;""-_+=?/|]", " ", e)
    return e

In [5]:
length = len(df)
new_df = df["content"]
while(length>=0):
    df["content"].iloc[length-1] = preprocessor(df["content"].iloc[length-1])
    length = length -1   

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.
# TODO
vectorizer =  CountVectorizer()

# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.
# TODO

content = df["content"].values
label = df["category"].values
msg_train, msg_test, label_train, label_test = train_test_split(content, label, test_size = 0.3)

# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.
# TODO
X_train = vectorizer.fit(msg_train)
X_train = vectorizer.transform(msg_train)
X_test = vectorizer.transform(msg_test)
# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
# TODO

lg = LogisticRegression()
lg.fit(X_train, label_train)

# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.
# TODO

predictions = lg.predict(X_test)

# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
# TODO
print(accuracy_score(label_test, predictions))
print(confusion_matrix(label_test, predictions))
print(classification_report(label_test,predictions))


0.9851708575112831
[[1085   17]
 [   6  443]]
              precision    recall  f1-score   support

         ham       0.99      0.98      0.99      1102
        spam       0.96      0.99      0.97       449

    accuracy                           0.99      1551
   macro avg       0.98      0.99      0.98      1551
weighted avg       0.99      0.99      0.99      1551



C:\Users\garvi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Step 4.

In [7]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.
features = vectorizer.get_feature_names_out()

# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
importance = lg.coef_[0]

# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.


In [43]:
check_mail = ["bout Yeag Technology Private Limited8 Platform | 21,400+ recruiters | 16,000+ Referrals | 1400+ Schools | 1.1 million+ Resume | 200+ Fellows. “Connecting Campus Students with Industrywith an aim to bring fundamental innovation in the education industry through artificial intelligence, data science and machine learning, We @Yeag Technology are continuously working towards creating data science integrated platform to provide various solution required to bring the said changes. Started in Jan 2016, we came a long way from where we began with an idea to various platform offering leading - edge solutions. - Strong mathematical skills to help collect, measure, organize and analyze data.We have openings for Data Science Engineer | - Intern | Trainee. Details are belowDesignation: Data Science EngineerExperience: FreshersLocation: Pan India / Depends on client LocationEligibility: B.Tech / B.sc/ BCA / or candidates with interest in CodingSalary Range:  LPA - 9LPA CTCSkillset Required- Experience in data mining.- Understanding of machine-learning and operations research.- Knowledge of R, SQL and Python; familiarity with Scala, Java or C++ is an asset.- Good knowledge of using business intelligence tools (e.g. Tableau) and data frameworks (e.g. Hadoop).- Analytical mind and business acumen.- Strong math skills (e.g. statistics, algebra).- Problem-solving aptitude.- Excellent communication and presentation skills.- BSc/BA in Computer Science, Engineering or relevant field; graduate degree in Data Science or other quantitative field is preferred.Interested one can schedule your telephonic Interview https://www.picktime.com/YeagTechnologyTelephonicConversationSchedulerIn case of any query you are free to contact with our team at +91-9540519995(WhatsApp).For updated jobs https://www.linkedin.com/showcase/42825656/WhatsApp Image 2022-07-08 at 4.47.20 PM.jpeg--You received this message because you are subscribed to the Google Groups MNC Jobs group.To unsubscribe from this group and stop receiving emails from it, send an email to mncjobindia+unsubscribe@googlegroups.com.To view this discussion on the web visit https://groups.google.com/d/msgid/mncjobindia/5b88249d-ded9-4809-b327-c7335d50e314n%40googlegroups.com."]

In [46]:
check_mail = ["FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"]

In [47]:
preprocessor(check_mail[0])

"freemsg hey there darling it's been 3 week's now and no word back  i'd like some fun you up for it still  tb ok  xxx std chgs to send, å£1.50 to rcv"

In [48]:
test = vectorizer.transform(check_mail)

In [49]:
predictions = lg.predict(test)

In [50]:
predictions

array(['spam'], dtype=object)

Submission
1. Upload the jupyter notebook to Forage.

All Done!